In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kedarsai/bird-species-classification-220-categories")

print("Path to dataset files:", path)

Path to dataset files: /home/c-enjalbert/.cache/kagglehub/datasets/kedarsai/bird-species-classification-220-categories/versions/1


In [2]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("chriamue/bird-species-classifier")
model = AutoModelForImageClassification.from_pretrained("chriamue/bird-species-classifier")

2025-06-16 20:08:50.313626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750097330.331374   45853 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750097330.336672   45853 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750097330.349887   45853 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750097330.349912   45853 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750097330.349913   45853 computation_placer.cc:177] computation placer alr

In [3]:
model

EfficientNetForImageClassification(
  (efficientnet): EfficientNetModel(
    (embeddings): EfficientNetEmbeddings(
      (padding): ZeroPad2d((0, 1, 0, 1))
      (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=valid, bias=False)
      (batchnorm): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
      (activation): SiLU()
    )
    (encoder): EfficientNetEncoder(
      (blocks): ModuleList(
        (0): EfficientNetBlock(
          (depthwise_conv): EfficientNetDepthwiseLayer(
            (depthwise_conv_pad): ZeroPad2d((0, 1, 0, 1))
            (depthwise_conv): EfficientNetDepthwiseConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same, groups=32, bias=False)
            (depthwise_norm): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
            (depthwise_act): SiLU()
          )
          (squeeze_excite): EfficientNetSqueezeExciteLayer(
            (squeeze): AdaptiveAvgPool2d(output

In [4]:
# Bird Species Classification Script
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import random
from transformers import AutoImageProcessor, AutoModelForImageClassification
from tqdm import tqdm

def load_model():
    """Load pre-trained bird species classifier from Hugging Face"""
    print("Loading bird species classifier model...")
    processor = AutoImageProcessor.from_pretrained("chriamue/bird-species-classifier")
    model = AutoModelForImageClassification.from_pretrained("chriamue/bird-species-classifier")
    model.eval()
    return processor, model

def predict_bird_species(image_path, processor, model):
    """
    Predict bird species from an image file
    
    Args:
        image_path (str): Path to the bird image
        processor: Image processor for the model
        model: Loaded bird classification model
    
    Returns:
        str: Predicted bird species
        float: Confidence score
    """
    # Load and preprocess image
    image = Image.open(image_path).convert("RGB")
    
    # Prepare image for the model
    inputs = processor(images=image, return_tensors="pt")
    
    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get probabilities
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Get predicted class
    predicted_class_idx = probs.argmax().item()
    predicted_label = model.config.id2label[predicted_class_idx]
    confidence = probs[0][predicted_class_idx].item()
    
    return predicted_label, confidence

def display_prediction(image_path, species, confidence):
    """Display image with prediction and confidence"""
    image = Image.open(image_path)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(image)
    plt.title(f"Predicted Species: {species}\nConfidence: {confidence:.2%}", fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def test_random_images(dataset_path, processor, model, num_images=5):
    """Test model on random images from dataset"""
    # Find image directories
    image_dirs = []
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_dirs.append(root)
                break
    
    image_dirs = list(set(image_dirs))  # Remove duplicates
    
    if not image_dirs:
        print(f"No image directories found in {dataset_path}")
        return
    
    print(f"Found {len(image_dirs)} species directories")
    
    # Select random species directories
    selected_dirs = random.sample(image_dirs, min(num_images, len(image_dirs)))
    
    results = []
    
    for directory in selected_dirs:
        # Get species name from directory
        species_name = os.path.basename(directory)
        
        # Get all images in directory
        image_files = [f for f in os.listdir(directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        if not image_files:
            continue
        
        # Select random image
        image_file = random.choice(image_files)
        image_path = os.path.join(directory, image_file)
        
        # Predict
        predicted_species, confidence = predict_bird_species(image_path, processor, model)
        
        # Store results
        results.append({
            'image_path': image_path,
            'true_species': species_name,
            'predicted_species': predicted_species,
            'confidence': confidence
        })
        
        # Display
        print(f"\nTrue species: {species_name}")
        display_prediction(image_path, predicted_species, confidence)
    
    # Calculate accuracy
    correct = sum(1 for r in results if r['predicted_species'].lower() == r['true_species'].lower())
    accuracy = correct / len(results) if results else 0
    print(f"\nAccuracy on random sample: {accuracy:.2%} ({correct}/{len(results)})")




In [5]:
# Define path to the dataset
dataset_path = "/home/c-enjalbert/Documents/EPSI/Architecture_IA(M.Mandon)/kedarsai_bird-species-classification-220-categories"

# Check if the path exists
if not os.path.exists(dataset_path):
    print(f"Dataset path {dataset_path} does not exist.")
    print("Please update the path to where you downloaded the dataset using kagglehub.")
    return

# Load model
processor, model = load_model()

# Option 1: Test on a specific image
# sample_image = "path/to/your/bird/image.jpg"
# species, confidence = predict_bird_species(sample_image, processor, model)
# display_prediction(sample_image, species, confidence)

# Option 2: Test on random images from dataset
print("Testing on random images from dataset...")
test_random_images(dataset_path, processor, model, num_images=5)

SyntaxError: 'return' outside function (4051413643.py, line 8)

In [ ]:
# Bird Species Batch Classification Script
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoImageProcessor, AutoModelForImageClassification
from concurrent.futures import ThreadPoolExecutor

def load_model():
    """Load pre-trained bird species classifier from Hugging Face"""
    print("Loading bird species classifier model...")
    processor = AutoImageProcessor.from_pretrained("chriamue/bird-species-classifier")
    model = AutoModelForImageClassification.from_pretrained("chriamue/bird-species-classifier")
    model.eval()
    return processor, model

def predict_single_image(args):
    """Helper function for parallel processing"""
    image_path, processor, model = args
    try:
        # Load and preprocess image
        image = Image.open(image_path).convert("RGB")
        
        # Prepare image for the model
        inputs = processor(images=image, return_tensors="pt")
        
        # Get prediction
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Get probabilities
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
        # Get top 3 predictions
        top_indices = torch.topk(probs, k=3).indices[0].tolist()
        top_probs = torch.topk(probs, k=3).values[0].tolist()
        
        # Get predicted labels
        predicted_labels = [model.config.id2label[idx] for idx in top_indices]
        
        return {
            'image_path': image_path,
            'top_predictions': predicted_labels,
            'confidences': top_probs
        }
    except Exception as e:
        return {
            'image_path': image_path,
            'error': str(e)
        }

def batch_predict(image_paths, processor, model, max_workers=4):
    """
    Process multiple images in batch using parallel processing
    
    Args:
        image_paths (list): List of paths to bird images
        processor: Image processor for the model
        model: Loaded bird classification model
        max_workers (int): Maximum number of parallel workers
    
    Returns:
        list: Prediction results for each image
    """
    print(f"Processing {len(image_paths)} images with {max_workers} workers...")
    results = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        args_list = [(path, processor, model) for path in image_paths]
        for result in tqdm(executor.map(predict_single_image, args_list), total=len(args_list)):
            results.append(result)
    
    return results

def find_all_images(base_dir):
    """Find all image files in directory and subdirectories"""
    image_paths = []
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(root, file))
    return image_paths

def visualize_batch_results(results, num_samples=5):
    """Visualize a sample of batch prediction results"""
    if not results:
        print("No results to visualize")
        return
    
    # Filter out errors
    valid_results = [r for r in results if 'error' not in r]
    if not valid_results:
        print("All results contain errors")
        return
    
    # Sample results to visualize
    sample_results = random.sample(valid_results, min(num_samples, len(valid_results)))
    
    # Create a grid of images
    fig, axes = plt.subplots(1, len(sample_results), figsize=(4*len(sample_results), 4))
    if len(sample_results) == 1:
        axes = [axes]
    
    for i, result in enumerate(sample_results):
        # Load image
        img = Image.open(result['image_path'])
        
        # Display image
        axes[i].imshow(img)
        axes[i].set_title(f"Top prediction:\n{result['top_predictions'][0]}\n{result['confidences'][0]:.1%}")
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

def save_results_to_csv(results, output_path):
    """Save batch prediction results to CSV file"""
    # Convert results to DataFrame
    data = []
    for result in results:
        if 'error' in result:
            data.append({
                'image_path': result['image_path'],
                'top_prediction': 'ERROR',
                'confidence': 0.0,
                'error': result['error']
            })
        else:
            data.append({
                'image_path': result['image_path'],
                'top_prediction': result['top_predictions'][0],
                'confidence': result['confidences'][0],
                'second_prediction': result['top_predictions'][1] if len(result['top_predictions']) > 1 else '',
                'second_confidence': result['confidences'][1] if len(result['confidences']) > 1 else 0.0,
                'third_prediction': result['top_predictions'][2] if len(result['top_predictions']) > 2 else '',
                'third_confidence': result['confidences'][2] if len(result['confidences']) > 2 else 0.0
            })
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")
    
    return df



In [ ]:
import random

# Define path to the dataset
dataset_path = "/home/c-enjalbert/Documents/EPSI/Architecture_IA(M.Mandon)/kedarsai_bird-species-classification-220-categories"

# Check if the path exists
if not os.path.exists(dataset_path):
    print(f"Dataset path {dataset_path} does not exist.")
    print("Please update the path to where you downloaded the dataset using kagglehub.")
# Load model
processor, model = load_model()

# Find all images or a subset
all_image_paths = find_all_images(dataset_path)
print(f"Found {len(all_image_paths)} images in the dataset.")

# Use a smaller sample for testing (optional)
sample_size = 100  # Adjust as needed
image_paths = random.sample(all_image_paths, min(sample_size, len(all_image_paths)))

# Process images in batch
results = batch_predict(image_paths, processor, model)

# Visualize some results
visualize_batch_results(results, num_samples=5)

# Save results to CSV
output_file = "bird_classification_results.csv"
df = save_results_to_csv(results, output_file)

# Print summary
print("\nClassification Results Summary:")
print(f"Total images processed: {len(results)}")
print(f"Images with errors: {sum(1 for r in results if 'error' in r)}")

Dataset path /home/c-enjalbert/Documents/EPSI/Architecture_IA(M.Mandon)/kedarsai_bird-species-classification-220-categories does not exist.
Please update the path to where you downloaded the dataset using kagglehub.
Loading bird species classifier model...
Found 0 images in the dataset.
Processing 0 images with 4 workers...


0it [00:00, ?it/s]

No results to visualize
Results saved to bird_classification_results.csv

Classification Results Summary:
Total images processed: 0
Images with errors: 0
